# **Homework**

The goal of this homework is to train a simple model for predicting the duration of a ride

## **Install**

In [2]:
# pip install pyarrow
# pip install fastparquet

## **Import**

In [3]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

## **Q1. Downloading the data**  

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".  

Download the data for January and February 2023.  

Read the data for January. How many columns are there?  

- 16
- 17
- 18
- 19

In [5]:
df = pd.read_parquet('/workspaces/MLOps_2024_Zoomcamp/data/yellow_tripdata_2023-01.parquet')

df.shape

(3066766, 19)

In [6]:
len(df.columns)

19

## **Q2. Computing duration**

Now let's compute the duration variable. It should contain the duration of a ride in minutes.  

What's the standard deviation of the trips duration in January?  

- 32.59
- 42.59
- 52.59
- 62.59

In [7]:
df["duration"] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
df["duration"] = df["duration"].apply(lambda td: td.total_seconds() / 60)

In [8]:
df["duration"].std()

42.59435124195458

## **Q3. Dropping outliers**

Next, we need to check the distribution of the duration variable.   
There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).  

What fraction of the records left after you dropped the outliers?  

- 90%
- 92%
- 95%
- 98%

In [9]:
df.shape

(3066766, 20)

In [10]:
nb_before = len(df)
nb_before

3066766

In [11]:
df = df[(df["duration"] >= 1) & (df["duration"] <= 60)]

nb_after = len(df)
nb_after

3009173

In [12]:
nb_after / nb_before * 100

98.1220282212598

## **Q4. One-hot encoding**

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.  

- Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
- Fit a dictionary vectorizer
- Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

- 2
- 155
- 345
- 515
- 715

In [13]:
dv = DictVectorizer()
features = ["PULocationID", "DOLocationID"]
df[features] = df[features].astype('str')

train_dicts = df[features].to_dict(orient='records')

dv= DictVectorizer(sparse= True)

X_train = dv.fit_transform(train_dicts)

In [14]:
X_train.shape

(3009173, 515)

In [15]:
dv.get_feature_names_out().shape[0]

515

## **Q5. Training a model**

Now let's use the feature matrix from the previous step to train a model.

- Train a plain linear regression model with default parameters, where duration is the response variable
- Calculate the RMSE of the model on the training data

What's the RMSE on train?

- 3.64
- 7.64
- 11.64
- 16.64

In [16]:
lr = LinearRegression()
y_train = df["duration"].values
lr.fit(X_train, y_train)

LinearRegression()

In [17]:
y_pred = lr.predict(X_train)

In [18]:
root_mean_squared_error(y_train, y_pred)

7.649261932106969

## **Q6. Evaluating the model**

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

- 3.81
- 7.81
- 11.81
- 16.81

In [19]:
features = ["PULocationID", "DOLocationID"]

def read_data(filename):
    df = pd.read_parquet(filename)

    df["duration"] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
    df["duration"] = df["duration"].apply(lambda td: td.total_seconds() / 60)

    df = df[(df["duration"] >= 1) & (df["duration"] <= 60)]

    df[features] = df[features].astype('str')

    return df

In [20]:
df_val = read_data("/workspaces/MLOps_2024_Zoomcamp/data/yellow_tripdata_2023-02.parquet")

In [21]:
test_dicts = df_val[features].to_dict(orient='records')
X_val = dv.transform(test_dicts)

In [22]:
y_val = df_val["duration"].values

In [23]:
y_pred = lr.predict(X_val)

In [24]:
root_mean_squared_error(y_val, y_pred)

7.811818743246608